In [ ]:
import pandas as pd
import numpy as np

CONFIG = {
    "widetable_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_path": r'./reverse_strategy_final.csv',
    # 选股条件不变（保留高胜率核心）
    "auction_ratio_threshold": 0.8,
    "industry_cold_rank": 0.5,
    "3d_loss_threshold": -10.0,
    # 优化后交易参数（提升盈亏比）
    "buy_down_ratio": 0.02,          # 买入价：前5日低点+2%（不变）
    "t1_stop_loss": -3.5,            # T+1收盘止损：从-3%收紧到-3.5%
    "t2_stop_loss": -3.5,            # T+2盘中止损：从-4%收紧到-3.5%
    "stop_profit": 2.5,              # 止盈线：从3%降到2.5%（更早锁利）
    "initial_fund": 100000.0
}

# 加载数据、选股函数与之前完全一致（无改动）
def load_data():
    df = pd.read_parquet(CONFIG["widetable_path"])
    df["date"] = pd.to_datetime(df["date"])
    df = df[df["paused"] == 0].copy()
    df["auction_vol_5d_mean"] = df.groupby("stock_code")["auc_volume"].transform(
        lambda x: x.rolling(5, min_periods=3).mean().shift(1).replace(0, 0.0001)
    )
    df["auc_volume_ratio"] = df["auc_volume"] / df["auction_vol_5d_mean"]
    df["daily_return"] = (df["close"] / df["pre_close"] - 1) * 100
    df["3d_return"] = df.groupby("stock_code")["daily_return"].transform(
        lambda x: x.rolling(3).sum().shift(1)
    )
    df["ma5"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(5).mean())
    df["ma20"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(20).mean())
    df["5d_low"] = df.groupby("stock_code")["low"].transform(
        lambda x: x.rolling(5).min().shift(1)
    )
    df["t1_target_low"] = df["5d_low"] * (1 + CONFIG["buy_down_ratio"])
    industry_rank = df.groupby(["date", "sw_l1_industry_name"])["daily_return"].mean().reset_index()
    industry_rank["industry_percentile"] = industry_rank.groupby("date")["daily_return"].rank(pct=True)
    df = df.merge(industry_rank[["date", "sw_l1_industry_name", "industry_percentile"]], 
                  on=["date", "sw_l1_industry_name"], how="left")
    return df

def select_reverse_stocks(df):
    all_selections = []
    valid_dates = sorted(df["date"].unique())
    for date in valid_dates:
        daily_df = df[df["date"] == date].copy()
        daily_df = daily_df[
            (daily_df["auc_volume_ratio"] <= CONFIG["auction_ratio_threshold"]) &
            (daily_df["ma5"] < daily_df["ma20"]) &
            (daily_df["3d_return"] <= CONFIG["3d_loss_threshold"]) &
            (daily_df["industry_percentile"] <= CONFIG["industry_cold_rank"]) &
            (daily_df["daily_return"].shift(1) >= -1.0)
        ].copy()
        if len(daily_df) == 0:
            continue
        daily_df["oversold_score"] = daily_df["3d_return"].rank(ascending=True)
        selected = daily_df.nsmallest(5, "oversold_score")
        selected["selection_date"] = date
        all_selections.append(selected[["selection_date", "stock_code", "t1_target_low", "5d_low"]])
    return pd.concat(all_selections, ignore_index=True)

# 回测函数（仅调整止损止盈阈值）
def backtest_reverse(selections, df):
    price_cols = ["open", "close", "low", "high", "pre_close"]
    price_map = df.set_index(["date", "stock_code"])[price_cols].to_dict("index")
    
    fund = CONFIG["initial_fund"]
    backtest_records = []
    
    for _, row in selections.iterrows():
        stock_code = row["stock_code"]
        t_date = row["selection_date"]
        t1_date = t_date + pd.Timedelta(days=1)
        t2_date = t1_date + pd.Timedelta(days=1)
        
        t1_data = price_map.get((t1_date, stock_code))
        if not t1_data:
            continue
        t1_open = t1_data["open"]
        t1_low = t1_data["low"]
        t1_close = t1_data["close"]
        target_buy_price = row["t1_target_low"]
        
        # 买入逻辑不变
        if t1_low <= target_buy_price <= t1_open:
            buy_price = target_buy_price
        elif t1_low <= target_buy_price:
            buy_price = t1_low
        else:
            continue
        
        shares = int((fund * 0.2) // (buy_price * 100)) * 100
        actual_buy_amount = shares * buy_price
        
        # 优化后止损止盈逻辑
        t1_return = (t1_close / buy_price - 1) * 100
        sell_date = t2_date
        sell_price = None
        
        if t1_return <= CONFIG["t1_stop_loss"]:
            # T+1收盘止损（从-3%→-3.5%）
            t2_data = price_map.get((t2_date, stock_code))
            sell_price = t2_data["open"] if t2_data else t1_close
            sell_type = "t1_close_stop_loss"
        else:
            t2_data = price_map.get((t2_date, stock_code))
            if not t2_data:
                sell_price = t1_close
                sell_type = "no_t2_data"
            else:
                t2_close = t2_data["close"]
                t2_low = t2_data["low"]
                t2_max_loss = (t2_low / buy_price - 1) * 100
                
                if t2_max_loss <= CONFIG["t2_stop_loss"]:
                    # T+2盘中止损（从-4%→-3.5%）
                    sell_price = buy_price * (1 + CONFIG["t2_stop_loss"] / 100)
                    sell_type = "t2_intraday_stop_loss"
                elif (t2_close / buy_price - 1) * 100 >= CONFIG["stop_profit"]:
                    # 止盈（从3%→2.5%）
                    sell_price = buy_price * (1 + CONFIG["stop_profit"] / 100)
                    sell_type = "stop_profit"
                else:
                    sell_price = t2_close
                    sell_type = "normal_sell"
        
        # 收益计算不变
        profit = (sell_price - buy_price) * shares
        fund += profit
        actual_return = (sell_price / buy_price - 1) * 100
        
        backtest_records.append({
            "t_date": t_date,
            "stock_code": stock_code,
            "buy_date": t1_date,
            "sell_date": sell_date,
            "buy_price": round(buy_price, 2),
            "sell_price": round(sell_price, 2),
            "return_rate": round(actual_return, 2),
            "profit": round(profit, 2),
            "fund_after": round(fund, 2),
            "sell_type": sell_type
        })
    
    backtest_df = pd.DataFrame(backtest_records)
    backtest_df.to_csv(CONFIG["backtest_path"], index=False, encoding="utf-8-sig")
    
    valid_returns = backtest_df["return_rate"].dropna()
    win_returns = valid_returns[valid_returns > 0]
    loss_returns = valid_returns[valid_returns <= 0]
    win_rate = len(win_returns) / len(valid_returns) * 100 if len(valid_returns) > 0 else 0
    profit_loss_ratio = abs(win_returns.mean() / loss_returns.mean()) if len(loss_returns) > 0 else 0
    
    print("="*60)
    print("🎯 最终版反向策略回测结果（高胜率+合理盈亏比）")
    print("="*60)
    print(f"初始资金：{CONFIG['initial_fund']:.2f}元 → 最终资金：{fund:.2f}元")
    print(f"累计收益率：{(fund / CONFIG['initial_fund'] - 1) * 100:.2f}%")
    print(f"总交易笔数：{len(valid_returns)} | 盈利笔数：{len(win_returns)} | 亏损笔数：{len(loss_returns)}")
    print(f"胜率：{win_rate:.2f}% | 盈亏比：{profit_loss_ratio:.2f}")
    print(f"平均盈利：{win_returns.mean():.2f}% | 平均亏损：{loss_returns.mean():.2f}%")
    print("="*60)
    
    return backtest_df

# 执行最终版策略
if __name__ == "__main__":
    df = load_data()
    selections = select_reverse_stocks(df)
    backtest_result = backtest_reverse(selections, df)

🎯 优化后反向策略回测结果（无未来函数+合理止损）
初始资金：100000.00元 → 最终资金：347925.55元
累计收益率：247.93%
总交易笔数：1339 | 盈利笔数：855 | 亏损笔数：484
胜率：63.85% | 盈亏比：0.90
平均盈利：2.24% | 平均亏损：-2.48%
